<div style="background-color:#000;"><img src="pqn.png"></img></div>

In [ ]:
import yfinance as yf
import statsmodels.api as sm

## Download and prepare stock data for analysis

We will download historical stock data for selected tickers from Yahoo Finance. We will also calculate their daily returns to prepare for further analysis.

In [ ]:
tickers = ['AAPL', 'MSFT', 'AMZN', 'NVDA', 'GOOG', 'META', 'TSLA', 'QQQ']
data = yf.download(tickers, start='2022-01-01', end='2023-12-31')['Adj Close']

In [ ]:
benchmark_returns = (
    data
    .pop("QQQ")
    .pct_change()
    .dropna()
)

In [ ]:
portfolio_returns = (
    data
    .pct_change()
    .dropna()
    .sum(axis=1)
)

We begin by defining a list of stock tickers that we are interested in analyzing. Using the yfinance library, we download the adjusted closing prices for these stocks over a specified time period. We separate the returns of the benchmark ETF, QQQ, from the portfolio of other stocks. We calculate the percentage change in adjusted closing prices for both the benchmark and the portfolio to get daily returns, which helps us understand the daily performance of these stocks.

## Visualize the returns

We will create plots to visualize the daily returns of both our portfolio and the benchmark.

In [ ]:
portfolio_returns.plot()
benchmark_returns.plot()

With the data prepared, we create visualizations of the daily returns for our portfolio and the benchmark. These plots help us to quickly understand the volatility and trends in the returns. The portfolio returns are the aggregated daily returns of all stocks except QQQ, while the benchmark returns come from the QQQ ETF alone. By plotting both, we can visually compare how the portfolio performs relative to the benchmark over time.

## Perform linear regression to analyze portfolio performance

We will use linear regression to determine the relationship between our portfolio returns and the benchmark returns. This will allow us to understand how our portfolio is performing relative to the market.

In [ ]:
def linreg(x, y):    
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    return model

In [ ]:
X = benchmark_returns.values
Y = portfolio_returns.values

In [ ]:
model = linreg(X, Y)
alpha, beta = model.params[0], model.params[1]

In [ ]:
print(model.summary())
print(f"Alpha: {alpha}")
print(f"Beta: {beta}")

We define a function to perform linear regression using the statsmodels library. We add a constant to include an intercept in the regression. This function returns a fitted model object that we can analyze. We input the benchmark returns as the independent variable and the portfolio returns as the dependent variable. The regression provides us with an alpha and beta value. Alpha represents the excess returns of the portfolio not explained by the benchmark, while beta measures the sensitivity of the portfolio to market movements. The summary of the model provides additional statistics to evaluate the fit.

## Build and analyze a hedged portfolio

We will create a hedged portfolio by adjusting the portfolio returns based on the beta from our regression. We will then analyze the performance of this hedged portfolio.

In [ ]:
hedged_portfolio_returns = -beta * benchmark_returns + portfolio_returns

In [ ]:
P = hedged_portfolio_returns.values
model = linreg(X, P)
alpha, beta = model.params[0], model.params[1]

In [ ]:
print(f"Alpha: {alpha}")
print(f"Beta: {round(beta, 6)}")

In [ ]:
hedged_portfolio_returns.plot()
benchmark_returns.plot()

Using the beta from our previous regression, we adjust the portfolio returns to create a hedged portfolio. This involves subtracting the product of beta and benchmark returns from the portfolio returns, effectively reducing market exposure. We perform another regression analysis on the hedged returns against the benchmark to evaluate the hedging effectiveness. The new alpha and beta values indicate how well the hedged portfolio performs independently of market movements. Finally, we plot the hedged portfolio returns alongside the benchmark to visually assess the results.

## Your next steps

Try changing the tickers in the code to analyze a different set of stocks. Observe how the alpha and beta values change with different portfolios. Experiment with different time periods to see how the market conditions affect your analysis.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.